In [ ]:
%pip install transformers langdetect deep-translator torch torchvision torchaudio langchain langchain-community langchain-ollama gradio -q

In [ ]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate

In [ ]:
model_name = "mistral"
llm = ChatOllama(
    temperature = 0,
    max_tokens = 2000,
    model = model_name
)

template = """
You're a native internal AI assistant. Help users with their important tasks, like a professor in a particular field.

Query: {query}
"""

prompt = PromptTemplate(
    input_variables = ["query"],
    template = template
)

In [ ]:
chain = prompt | llm

In [ ]:
from langdetect import detect
from deep_translator import GoogleTranslator

class Translation:
  def __init__(self, text, destination):
    self.text = text
    self.destination = destination

    try:
      self.original = detect(self.text)
    except Exception as error:
      print(f"Error Occurred, Details : {error}")
      self.original = "auto"

  def translate(self):
    translator = GoogleTranslator(source=self.original, target=self.destination)
    translation = translator.translate(self.text)

    return translation

In [ ]:
import gradio as gr

In [26]:
def reply(message, history):
  txt = Translation(message, "en")

  if txt.original == "en":
    response = chain.invoke({
        "query": message
    })

    return response
  else:
    translation = txt.translate()

    response = chain.invoke({
        "query": translation
    })

    t = Translation(response.content, txt.original)
    final_response = t.translate()

    return final_response

In [ ]:
question = "What is Deep Learning?"

result = reply(question, history = [])

In [ ]:
result.content

In [27]:
question = "ரியல் எஸ்டேட்டில் அதிக பணம் சம்பாதிப்பது எப்படி?"

result = reply(question, history = [])

result

'ரியல் எஸ்டேட் துறையில் உங்கள் வருமானத்தை அதிகரிக்க, பின்வரும் உத்திகளைக் கவனியுங்கள்:\n\n1. ரியல் எஸ்டேட்டில் முதலீடு செய்யுங்கள்: சொத்துக்களை வாங்கி, செயலற்ற வருமானத்திற்காக வாடகைக்கு விடுங்கள். சந்தை மதிப்பு அதிகரித்தால் நீங்கள் சொத்துக்களை லாபத்தில் விற்கலாம்.\n\n2. ரியல் எஸ்டேட் மேம்பாடு: புதிய வீடுகளைக் கட்டுவதன் மூலமோ அல்லது ஏற்கனவே உள்ளவற்றைப் புதுப்பித்து அதிக விலைக்கு விற்பதன் மூலமோ சொத்துக்களை அபிவிருத்தி செய்யுங்கள். இதற்கு அதிக மூலதனம் தேவை, ஆனால் அதிக வருமானம் கிடைக்கும்.\n\n3. ரியல் எஸ்டேட் புரட்டல்: குறைவாக வாங்கவும், புதுப்பிக்கவும், அதிகமாக விற்கவும். இந்த மூலோபாயம் பழுதுபார்ப்பு அல்லது புதுப்பிப்புகள் தேவைப்படும் சொத்துக்களை வாங்குதல், அவற்றை சரிசெய்தல் மற்றும் லாபத்திற்காக விரைவாக விற்பது ஆகியவை அடங்கும்.\n\n4. ரியல் எஸ்டேட் மொத்த விற்பனை: குறைவான மதிப்புள்ள சொத்துகளைக் கண்டறிந்து, அவற்றை வாங்குவதற்கான ஒப்பந்தத்தை பேச்சுவார்த்தை நடத்தவும், பின்னர் ஒப்பந்தத்தை மற்றொரு முதலீட்டாளருக்கு கட்டணமாக வழங்கவும்.\n\n5. ரியல் எஸ்டேட் மேலாண்மை: கட்டணத்திற்கு ஈடாக மற்றவர்களுக்கு

In [ ]:
demo = gr.ChatInterface(fn=reply, title = "Multi-Lingual ChatBot")

demo.launch(debug=False, share=False)